# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [30]:
name = 'yannis'

## Imports

In [31]:
import pandas as pd
import numpy as np
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import * 

In [32]:
'''# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])'''

"# config parser\nimport configparser\n\n# get the rounding_digits from the ini file\nconfig = configparser.ConfigParser()\nconfig.read('../config.ini')\nrounding_digits = int(config['DEFAULT']['rounding_digits'])"

## Constants

In [ ]:
personal_data = personal_data_provider(name)
E_11 = personal_data[0]
E_22 = personal_data[1]
G_12 = personal_data[2]
nu_12 = personal_data[3]
sigma_u_c = 650
knockdown = 0.9
print(f"Your personal data is: E11 = {E_11}, E22 = {E_22}, G12 = {G_12}, nu12 = {nu_12}. Please verify.")

#Stacking sequences 
#Stacking sequences 
panelStack=[45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]
StringerFlange=[45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]
StringerWeb=[-45,-45,45,45,0,0,90,90,90,90,0,0,45,45,-45,-45]

#Ply thicknesses 
tPanel = 0.552
tStringer = 0.25

#Geometric dimensions 
stringer_pitch = 400
effective_width = stringer_pitch/2
panelwidth = 200

panel_element_length = 250
stringer_element_length = 250


Your personal data is: E11 = 119325.52800000002, E22 = 9178.884, G12 = 4589.442, nu12 = 0.33. Please verify.


# Here we need to calculate the necessary ABD information 

In [ ]:
# Get ABD matrix information 
ABD_panel, ABD_panel_inverse = calculateABD(stacksequence=panelStack, plyT=tPanel, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12)
ABD_flange, ABD_flange_inverse = calculateABD(stacksequence=StringerFlange, plyT=tStringer, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12)
ABD_web, ABD_web_inverse = calculateABD(stacksequence=StringerWeb, plyT=tStringer, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12)

ABD_panel = ABD_panel * knockdown
ABD_panel_inverse = ABD_panel_inverse * knockdown

ABD_flange = ABD_flange * knockdown
ABD_flange_inverse = ABD_flange_inverse * knockdown

ABD_web = ABD_web * knockdown
ABD_web_inverse = ABD_web_inverse * knockdown
#Compute necessary axial Moduli 
E_x_skin = ABD_panel[0][0]/tStringer
E_x_flange = ABD_flange[0][0]/tStringer
E_x_web = 1/(ABD_web_inverse[0][0]*tStringer)


#Compute necessary bending around y Moduli 
E_y_skin = ABD_panel[3][3] *12/tPanel**3
E_y_flange = ABD_flange[3][3]*12/tStringer**2
E_y_web = E_x_web


# Importing necessary files 

## Import everything for panels 

In [ ]:
# Import panel properties and match the elements to the respective panel
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
thickness = [8.832]* len(paneldf)
paneldf['thickness'] = thickness
paneldf = paneldf.rename(columns={'Elements':'Element ID', 'XX':'sigmaXX', 'Loadcase':'Load Case'})
paneldf['Component Name'] = paneldf.apply(elementComponentMatch, axis=1)
#panelPropertiesdf = pd.read_csv(f'../data/{name}/properties/panel_properties.csv', index_col=0)
#panelPropertiesdf = panelPropertiesdf.drop(['mass', 'Component Name'], axis=1)
#paneldf = pd.merge(paneldf, panelPropertiesdf, on='Element ID', how='left', suffixes=('_caller', '_other'))
paneldf = paneldf.drop(['XY', 'YY', 'Layer', 'Step', 'FileID'], axis=1)
paneldf.head(5)

,Element ID,Load Case,sigmaXX,thickness,Component Name
0,1,1,-41.655796,4,panel1
1,2,1,-50.504929,4,panel1
2,3,1,-41.692043,4,panel1
3,4,1,-41.826157,4,panel1
4,5,1,-38.997730,4,panel1


## Add a volume column to the panels 

In [36]:
paneldf['Volume'] = paneldf.apply(panel_element_volume, elementLength=panel_element_length, elementWidth=panelwidth, axis=1)
paneldf

,Element ID,Load Case,sigmaXX,thickness,Component Name,Volume
0,1,1,-41.655796,4,panel1,200000
1,2,1,-50.504929,4,panel1,200000
2,3,1,-41.692043,4,panel1,200000
3,4,1,-41.826157,4,panel1,200000
4,5,1,-38.997730,4,panel1,200000
...,...,...,...,...,...,...
85,26,3,-12.134929,4,panel5,200000
86,27,3,-14.113474,4,panel5,200000
87,28,3,-9.420760,4,panel5,200000
88,29,3,-14.460136,4,panel5,200000


## Extract thicknesses left and right 

In [37]:
leftThickness = []
rightThickness = []
for i in range(0,4):
    leftThickness.append(paneldf['thickness'][0+3*i])
    rightThickness.append(paneldf['thickness'][6+3*i])
print(leftThickness)
print(rightThickness)

[np.int64(4), np.int64(4), np.int64(4), np.int64(4)]
[np.int64(4), np.int64(4), np.int64(4), np.int64(4)]


## Import everything for stringers 

In [38]:
# Open and match stringer properties 
stringerdf = pd.read_csv(f'../data/{name}/stringer.csv')
stringerdf = stringerdf.rename(columns={'Elements':'Element ID', 'Element Stresses (1D):CBAR/CBEAM Axial Stress':'sigmaXX', 'Loadcase':'Load Case'})
stringerdf['Component Name'] = stringerdf.apply(elementComponentMatch, axis=1)
dflength = len(stringerdf)
stringerdf['dim1'] = [70] * dflength
stringerdf['dim2'] = [44] * dflength
stringerdf['dim3'] = [4] * dflength
stringerdf['dim4'] = [4] * dflength
# Now add the stringer properties '../data/{name}/stringer_properties.csv
#stringerPropertiesdf = pd.read_csv(f'../data/{name}/properties/stringer_properties.csv', index_col=0)
#stringerPropertiesdf = stringerPropertiesdf.reset_index()
#stringerPropertiesdf.rename(columns={'beamsects': 'Component Name', 'beamsect_dim1': 'dim1', 'beamsect_dim2': 'dim2', 'beamsect_dim3': 'dim3', 'beamsect_dim4': 'dim4'}, inplace=True)
# Add "stringer" prefix to Component Name
#stringerPropertiesdf['Component Name'] = 'stringer' + stringerPropertiesdf['Component Name'].astype(str)
# Merge the dataframes
#stringerdf = pd.merge(stringerdf, stringerPropertiesdf, on='Component Name', how='left', suffixes=('_caller', '_other'))
stringerdf = stringerdf.drop(['FileID', 'Step', ], axis=1)
stringerdf

,Element ID,Load Case,sigmaXX,Component Name,dim1,dim2,dim3,dim4
0,40,1,-68.654549,stringer1,70,44,4,4
1,41,1,-84.363640,stringer1,70,44,4,4
2,42,1,-69.236366,stringer1,70,44,4,4
3,46,1,-71.127274,stringer2,70,44,4,4
4,47,1,-88.000000,stringer2,70,44,4,4
5,48,1,-70.836365,stringer2,70,44,4,4
6,52,1,-71.563637,stringer3,70,44,4,4
7,53,1,-88.290909,stringer3,70,44,4,4
8,54,1,-70.690910,stringer3,70,44,4,4
9,58,1,-69.236366,stringer4,70,44,4,4


## Add volume to the stringer elements

In [39]:
stringerdf['Volume']= stringerdf.apply(stringer_element_volume, elementLength = stringer_element_length, axis=1)
stringerdf.head(3)

,Element ID,Load Case,sigmaXX,Component Name,dim1,dim2,dim3,dim4,Volume
0,40,1,-68.654549,stringer1,70,44,4,4,110000
1,41,1,-84.363640,stringer1,70,44,4,4,110000
2,42,1,-69.236366,stringer1,70,44,4,4,110000


# Now we rearrange the structure a bit

## First we split the 3 loadcases 

In [40]:
loadCase1dfPanel = paneldf[paneldf["Load Case"] == 1]
loadCase2dfPanel = paneldf[paneldf["Load Case"] == 2]
loadCase3dfPanel = paneldf[paneldf["Load Case"] == 3]
loadCase1dfStringer = stringerdf[stringerdf["Load Case"] == 1]
loadCase2dfStringer = stringerdf[stringerdf["Load Case"] == 2]
loadCase3dfStringer = stringerdf[stringerdf["Load Case"] == 3]
loadCase1dfPanel.head(5)
loadCase1dfStringer.head(5)

,Element ID,Load Case,sigmaXX,Component Name,dim1,dim2,dim3,dim4,Volume
0,40,1,-68.654549,stringer1,70,44,4,4,110000
1,41,1,-84.363640,stringer1,70,44,4,4,110000
2,42,1,-69.236366,stringer1,70,44,4,4,110000
3,46,1,-71.127274,stringer2,70,44,4,4,110000
4,47,1,-88.000000,stringer2,70,44,4,4,110000


# Now we need to combine the panels and the stringers

In [41]:
# For every loadcase
# Efficiently combine panels for load case 1 
panel_groups_1 = []
for i in range(1, 5):
    df1 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i}'].copy()
    df1 = df1[df1['Element ID'].isin(np.arange(df1['Element ID'].min()+3, df1['Element ID'].min()+6))]
    df2 = loadCase1dfPanel[loadCase1dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df2 = df2[df2['Element ID'].isin(np.arange(df2['Element ID'].min(), df2['Element ID'].min()+3))]
    df3 = loadCase1dfStringer[loadCase1dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_1.extend([df1, df2, df3])
lc1combined = pd.concat(panel_groups_1, ignore_index=True)
lc1combined.head(10)

,Element ID,Load Case,sigmaXX,thickness,Component Name,Volume,Stiffener,dim1,dim2,dim3,dim4
0,4,1,-41.826157,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
1,5,1,-38.997730,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
2,6,1,-41.510414,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
3,7,1,-40.618355,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
4,8,1,-38.335014,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
5,9,1,-39.807552,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
6,40,1,-68.654549,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
7,41,1,-84.363640,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
8,42,1,-69.236366,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
9,10,1,-39.283321,4.0,panel2,200000,stiffener2,NaN,NaN,NaN,NaN


In [42]:
# Efficiently combine panels for load case 2
panel_groups_2 = []
for i in range(1, 5):
    df1 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i}'].copy()
    df1 = df1[df1['Element ID'].isin(np.arange(df1['Element ID'].min()+3, df1['Element ID'].min()+6))]
    df2 = loadCase2dfPanel[loadCase2dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df2 = df2[df2['Element ID'].isin(np.arange(df2['Element ID'].min(), df2['Element ID'].min()+3))]
    df3 = loadCase2dfStringer[loadCase2dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_2.extend([df1, df2, df3])
lc2combined = pd.concat(panel_groups_2, ignore_index=True)
lc2combined.head(10)

,Element ID,Load Case,sigmaXX,thickness,Component Name,Volume,Stiffener,dim1,dim2,dim3,dim4
0,4,2,-41.115955,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
1,5,2,-38.765289,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
2,6,2,-41.349415,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
3,7,2,-40.391071,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
4,8,2,-38.068726,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
5,9,2,-39.536495,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
6,40,2,-68.945457,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
7,41,2,-86.981819,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
8,42,2,-69.527275,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
9,10,2,-39.929279,4.0,panel2,200000,stiffener2,NaN,NaN,NaN,NaN


In [43]:
# Efficiently combine panels for load case 3 
panel_groups_3 = []
for i in range(1, 5):
    df1 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i}'].copy()
    df1 = df1[df1['Element ID'].isin(np.arange(df1['Element ID'].min()+3, df1['Element ID'].min()+6))]
    df2 = loadCase3dfPanel[loadCase3dfPanel['Component Name'] == f'panel{i+1}'].copy()
    df2 = df2[df2['Element ID'].isin(np.arange(df2['Element ID'].min(), df2['Element ID'].min()+3))]
    df3 = loadCase3dfStringer[loadCase3dfStringer['Component Name']==f'stringer{i}'].copy()
    df1['Stiffener'] = 'stiffener'+str(i)
    df2['Stiffener'] = 'stiffener'+str(i)
    df3['Stiffener'] = 'stiffener'+str(i)
    panel_groups_3.extend([df1, df2, df3])
lc3combined = pd.concat(panel_groups_3, ignore_index=True)
lc3combined.head(10)

,Element ID,Load Case,sigmaXX,thickness,Component Name,Volume,Stiffener,dim1,dim2,dim3,dim4
0,4,3,-13.440712,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
1,5,3,-13.968595,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
2,6,3,-12.605365,4.0,panel1,200000,stiffener1,NaN,NaN,NaN,NaN
3,7,3,-15.165724,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
4,8,3,-11.693856,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
5,9,3,-11.849609,4.0,panel2,200000,stiffener1,NaN,NaN,NaN,NaN
6,40,3,-22.109091,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
7,41,3,-27.927273,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
8,42,3,-24.436363,NaN,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
9,10,3,-10.733357,4.0,panel2,200000,stiffener2,NaN,NaN,NaN,NaN


In [44]:
# We fill the empty spaces with zero
lc1combined = lc1combined.fillna(0)
lc2combined = lc2combined.fillna(0)
lc3combined = lc3combined.fillna(0)
lc1combined

,Element ID,Load Case,sigmaXX,thickness,Component Name,Volume,Stiffener,dim1,dim2,dim3,dim4
0,4,1,-41.826157,4.0,panel1,200000,stiffener1,0.0,0.0,0.0,0.0
1,5,1,-38.997730,4.0,panel1,200000,stiffener1,0.0,0.0,0.0,0.0
2,6,1,-41.510414,4.0,panel1,200000,stiffener1,0.0,0.0,0.0,0.0
3,7,1,-40.618355,4.0,panel2,200000,stiffener1,0.0,0.0,0.0,0.0
4,8,1,-38.335014,4.0,panel2,200000,stiffener1,0.0,0.0,0.0,0.0
5,9,1,-39.807552,4.0,panel2,200000,stiffener1,0.0,0.0,0.0,0.0
6,40,1,-68.654549,0.0,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
7,41,1,-84.363640,0.0,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
8,42,1,-69.236366,0.0,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0
9,10,1,-39.283321,4.0,panel2,200000,stiffener2,0.0,0.0,0.0,0.0


# Now we can aggregate the loadcases according to stiffeners

## Multiply volume and stress together for averaging 

In [45]:
lc1combined['XX * Volume'] = lc1combined['sigmaXX'] * lc1combined['Volume']
lc2combined['XX * Volume'] = lc2combined['sigmaXX'] * lc2combined['Volume']
lc3combined['XX * Volume'] = lc3combined['sigmaXX'] * lc3combined['Volume']
lc1combined.head(9)

,Element ID,Load Case,sigmaXX,thickness,Component Name,Volume,Stiffener,dim1,dim2,dim3,dim4,XX * Volume
0,4,1,-41.826157,4.0,panel1,200000,stiffener1,0.0,0.0,0.0,0.0,-8.365231e+06
1,5,1,-38.997730,4.0,panel1,200000,stiffener1,0.0,0.0,0.0,0.0,-7.799546e+06
2,6,1,-41.510414,4.0,panel1,200000,stiffener1,0.0,0.0,0.0,0.0,-8.302083e+06
3,7,1,-40.618355,4.0,panel2,200000,stiffener1,0.0,0.0,0.0,0.0,-8.123671e+06
4,8,1,-38.335014,4.0,panel2,200000,stiffener1,0.0,0.0,0.0,0.0,-7.667003e+06
5,9,1,-39.807552,4.0,panel2,200000,stiffener1,0.0,0.0,0.0,0.0,-7.961510e+06
6,40,1,-68.654549,0.0,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0,-7.552000e+06
7,41,1,-84.363640,0.0,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0,-9.280000e+06
8,42,1,-69.236366,0.0,stringer1,110000,stiffener1,70.0,44.0,4.0,4.0,-7.616000e+06


## Load case 1 

In [46]:
lc1combined = lc1combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc1combined['sigma_XX_avg'] = lc1combined['XX * Volume'] / lc1combined['Volume']
lc1combined['tLeft'] = leftThickness
lc1combined['tRight'] = rightThickness
lc1combined = lc1combined.drop(['XX * Volume'], axis=1)
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,1530000,70.0,44.0,4.0,4.0,-47.494801,4,4
stiffener2,1530000,70.0,44.0,4.0,4.0,-47.414490,4,4
stiffener3,1530000,70.0,44.0,4.0,4.0,-47.581302,4,4
stiffener4,1530000,70.0,44.0,4.0,4.0,-47.275669,4,4


## Load case 2

In [47]:
lc2combined = lc2combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc2combined['sigma_XX_avg'] = lc2combined['XX * Volume'] / lc2combined['Volume']
lc2combined = lc2combined.drop(['XX * Volume'], axis=1)
lc2combined['tLeft'] = leftThickness
lc2combined['tRight'] = rightThickness
lc2combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,1530000,70.0,44.0,4.0,4.0,-47.480648,4,4
stiffener2,1530000,70.0,44.0,4.0,4.0,-47.096355,4,4
stiffener3,1530000,70.0,44.0,4.0,4.0,-47.346997,4,4
stiffener4,1530000,70.0,44.0,4.0,4.0,-47.518229,4,4


## Load case 3

In [48]:
lc3combined = lc3combined.groupby('Stiffener').agg({
    'XX * Volume':'sum',
    'Volume':'sum',
    'dim1': 'max',
    'dim2': 'max',
    'dim3': 'max',
    'dim4': 'max',
})
lc3combined['sigma_XX_avg'] = lc3combined['XX * Volume'] / lc3combined['Volume']
lc3combined = lc3combined.drop(['XX * Volume'], axis=1)
lc3combined['tLeft'] = leftThickness
lc3combined['tRight'] = rightThickness
lc3combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight
Stiffener,,,,,,,,
stiffener1,1530000,70.0,44.0,4.0,4.0,-15.644949,4,4
stiffener2,1530000,70.0,44.0,4.0,4.0,-15.420775,4,4
stiffener3,1530000,70.0,44.0,4.0,4.0,-16.325813,4,4
stiffener4,1530000,70.0,44.0,4.0,4.0,-15.674643,4,4


# Now we add Cross-Section Properties of the combined skin and hat stringer crosssection 

In [49]:
# Load case 1
lc1combined[['I_yy', 'areaTot', 'EI_comb', 'E_y_comb', 'z_bar']] = lc1combined.apply(crosssectional_properties_tee_skin_row, stringer_pitch=stringer_pitch,
                                                                    E_x_skin=E_x_skin, E_x_flange=E_x_flange, E_x_web=E_x_web,
                                                                    E_y_skin=E_y_skin, E_y_flange=E_y_flange, E_y_web=E_y_web,
                                                                    axis=1, result_type='expand')
# Load case 2
lc2combined[['I_yy', 'areaTot', 'EI_comb', 'E_y_comb', 'z_bar']] = lc2combined.apply(crosssectional_properties_tee_skin_row, stringer_pitch=stringer_pitch,
                                                                    E_x_skin=E_x_skin, E_x_flange=E_x_flange, E_x_web=E_x_web,
                                                                    E_y_skin=E_y_skin, E_y_flange=E_y_flange, E_y_web=E_y_web,
                                                                    axis=1, result_type='expand')
# Load case 3
lc3combined[['I_yy', 'areaTot', 'EI_comb', 'E_y_comb', 'z_bar']] = lc3combined.apply(crosssectional_properties_tee_skin_row, stringer_pitch=stringer_pitch,
                                                                    E_x_skin=E_x_skin, E_x_flange=E_x_flange, E_x_web=E_x_web,
                                                                    E_y_skin=E_y_skin, E_y_flange=E_y_flange, E_y_web=E_y_web,
                                                                    axis=1, result_type='expand')
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,EI_comb,E_y_comb,z_bar
Stiffener,,,,,,,,,,,,,
stiffener1,1530000,70.0,44.0,4.0,4.0,-47.494801,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011
stiffener2,1530000,70.0,44.0,4.0,4.0,-47.414490,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011
stiffener3,1530000,70.0,44.0,4.0,4.0,-47.581302,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011
stiffener4,1530000,70.0,44.0,4.0,4.0,-47.275669,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011


# Now we calculate the columnbuckling with Euler Johnson 

In [50]:
# Load case 1EModulus, DIM1, DIM2, DIM3, sigma_yield, r
lc1combined['sigma_crip'] = lc1combined.apply(lambda row: sigma_crip(sigma_u_c=sigma_u_c, DIM2=row['dim2'], DIM3=row['dim3'],DIM4=row['dim4'], r=0 ),
                                                axis=1)
# Load case 2
lc2combined['sigma_crip'] = lc2combined.apply(lambda row: sigma_crip(sigma_u_c=sigma_u_c, DIM2=row['dim2'], DIM3=row['dim3'],DIM4=row['dim4'], r=0 ),
                                                axis=1)
#Load case 3
lc3combined['sigma_crip'] = lc3combined.apply(lambda row: sigma_crip(sigma_u_c=sigma_u_c, DIM2=row['dim2'], DIM3=row['dim3'],DIM4=row['dim4'], r=0 ),
                                                axis=1)
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,EI_comb,E_y_comb,z_bar,sigma_crip
Stiffener,,,,,,,,,,,,,,
stiffener1,1530000,70.0,44.0,4.0,4.0,-47.494801,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953
stiffener2,1530000,70.0,44.0,4.0,4.0,-47.414490,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953
stiffener3,1530000,70.0,44.0,4.0,4.0,-47.581302,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953
stiffener4,1530000,70.0,44.0,4.0,4.0,-47.275669,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953


## Add lambda_crit to the loadcases

In [51]:
lc1combined['lambda_crit'] = lc1combined.apply(lambda row: lambda_crit(EModulus=row['E_y_comb'], sigma_crip=row['sigma_crip']), axis=1)
lc2combined['lambda_crit'] = lc2combined.apply(lambda row: lambda_crit(EModulus=row['E_y_comb'], sigma_crip=row['sigma_crip']), axis=1)
lc3combined['lambda_crit'] = lc3combined.apply(lambda row: lambda_crit(EModulus=row['E_y_comb'], sigma_crip=row['sigma_crip']), axis=1)

## Add lambda to the loadcases

In [52]:
lc1combined['lambda'] = lc1combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc2combined['lambda'] = lc2combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)
lc3combined['lambda'] = lc3combined.apply(lambda row: lmd(row['I_yy'], row['areaTot'], stringer_element_length*3), axis=1)

## Add radius of gyration to the loadcases

In [53]:
lc1combined['r_gyr'] = lc1combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc2combined['r_gyr'] = lc2combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)
lc3combined['r_gyr'] = lc3combined.apply(lambda row: r_gyr(row['I_yy'], row['areaTot']), axis=1)

## Now calculate the critical stress & Reserve Factor

In [54]:
lc1combined[['sigma_crit', 'Reserve Factor']] = lc1combined.apply(chooseBuckling, axis=1, result_type='expand')
lc2combined[['sigma_crit', 'Reserve Factor']] = lc2combined.apply(chooseBuckling, axis=1, result_type='expand')
lc3combined[['sigma_crit', 'Reserve Factor']] = lc3combined.apply(chooseBuckling, axis=1, result_type='expand')
lc1combined

,Volume,dim1,dim2,dim3,dim4,sigma_XX_avg,tLeft,tRight,I_yy,areaTot,EI_comb,E_y_comb,z_bar,sigma_crip,lambda_crit,lambda,r_gyr,sigma_crit,Reserve Factor
Stiffener,,,,,,,,,,,,,,,,,,,
stiffener1,1530000,70.0,44.0,4.0,4.0,-47.494801,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953,634.011332,95.215458,7.876872,200.990545,2.821229
stiffener2,1530000,70.0,44.0,4.0,4.0,-47.414490,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953,634.011332,95.215458,7.876872,200.990545,2.826007
stiffener3,1530000,70.0,44.0,4.0,4.0,-47.581302,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953,634.011332,95.215458,7.876872,200.990545,2.816100
stiffener4,1530000,70.0,44.0,4.0,4.0,-47.275669,4,4,126572.037878,2040.0,5.239659e+11,4.139666e+06,-0.769011,203.282953,634.011332,95.215458,7.876872,200.990545,2.834306


# Cleanup data for output 

## Drop unenessacry columns 

In [55]:
lc1combined = lc1combined.drop(['Volume', 'tLeft', 'tRight', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'sigma_crit', 'E_y_comb', 'I_yy'], axis=1)
lc2combined = lc2combined.drop(['Volume',  'tLeft', 'tRight', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'sigma_crit', 'E_y_comb', 'I_yy'], axis=1)
lc3combined = lc3combined.drop(['Volume',  'tLeft', 'tRight', 'dim1', 'dim2', 'dim3', 'dim4', 'areaTot', 'sigma_crit', 'E_y_comb', 'I_yy'], axis=1)
lc1combined

,sigma_XX_avg,EI_comb,z_bar,sigma_crip,lambda_crit,lambda,r_gyr,Reserve Factor
Stiffener,,,,,,,,
stiffener1,-47.494801,5.239659e+11,-0.769011,203.282953,634.011332,95.215458,7.876872,2.821229
stiffener2,-47.414490,5.239659e+11,-0.769011,203.282953,634.011332,95.215458,7.876872,2.826007
stiffener3,-47.581302,5.239659e+11,-0.769011,203.282953,634.011332,95.215458,7.876872,2.816100
stiffener4,-47.275669,5.239659e+11,-0.769011,203.282953,634.011332,95.215458,7.876872,2.834306


## ROUND & Add together the load cases 

In [56]:
# Rename colums for concat
lc1combined = lc1combined.rename(columns={'sigma_XX_avg':'XX_avg_LC1', 'sigma_crit':'sigma_crit_LC1', 'Reserve Factor':'RF_LC1', 'sigma_crip':'sigma_crip_LC1'})
lc2combined = lc2combined.rename(columns={'sigma_XX_avg':'XX_avg_LC2', 'sigma_crit':'sigma_crit_LC2', 'Reserve Factor':'RF_LC2', 'sigma_crip':'sigma_crip_LC2'})
lc3combined = lc3combined.rename(columns={'sigma_XX_avg':'XX_avg_LC3', 'sigma_crit':'sigma_crit_LC3', 'Reserve Factor':'RF_LC3', 'sigma_crip':'sigma_crip_LC3'})

In [57]:
outputdf = pd.concat([lc1combined,lc2combined,lc3combined], axis = 1)
#outputdf = outputdf.round(rounding_digits)
# After concatenation, keep only the first column of cross section propertries and drop the rest
outputdf['Lambda'] = outputdf.filter(like='lambda').iloc[:, 1]  # Take the first lambda column
outputdf['Lambda_crit'] = outputdf.filter(like='lambda_crit').iloc[:, 0]  # Take the first I_yy column
outputdf['R_gyr'] = outputdf.filter(like='r_gyr').iloc[:, 0]  # Take the first I_yy column
outputdf['EI_Comb'] = outputdf.filter(like='EI_comb').iloc[:, 0]  # Take the first I_yy column
outputdf['z_EC,comb'] = outputdf.filter(like='EI_comb').iloc[:, 0]  # Take the first I_yy column
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('lambda')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('r_gyr')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('EI_comb')])
outputdf = outputdf.drop(columns=[col for col in outputdf.columns if col.startswith('z_bar')])
outputdf.head(10)

,XX_avg_LC1,sigma_crip_LC1,RF_LC1,XX_avg_LC2,sigma_crip_LC2,RF_LC2,XX_avg_LC3,sigma_crip_LC3,RF_LC3,Lambda,Lambda_crit,R_gyr,EI_Comb,"z_EC,comb"
Stiffener,,,,,,,,,,,,,,
stiffener1,-47.494801,203.282953,2.821229,-47.480648,203.282953,2.822070,-15.644949,203.282953,8.564662,95.215458,634.011332,7.876872,5.239659e+11,5.239659e+11
stiffener2,-47.414490,203.282953,2.826007,-47.096355,203.282953,2.845097,-15.420775,203.282953,8.689168,95.215458,634.011332,7.876872,5.239659e+11,5.239659e+11
stiffener3,-47.581302,203.282953,2.816100,-47.346997,203.282953,2.830036,-16.325813,203.282953,8.207475,95.215458,634.011332,7.876872,5.239659e+11,5.239659e+11
stiffener4,-47.275669,203.282953,2.834306,-47.518229,203.282953,2.819838,-15.674643,203.282953,8.548437,95.215458,634.011332,7.876872,5.239659e+11,5.239659e+11


## Add other requested values 

In [58]:
outputLength = len(outputdf)
outputdf['E_hom,b,flange'] = [E_y_flange] * outputLength
outputdf['E_hom,b,web'] = [E_y_web] * outputLength
outputdf['E_hom,b,skin_left'] = [E_y_skin] * outputLength
outputdf['E_hom,b,skin_right'] = [E_y_skin] * outputLength


# Generate output file 

In [59]:
outputdf.to_excel(f'../data/{name}/output/processed_f.xlsx')